In [5]:
import mindspore.nn as nn
from mindspore import ops as ops
from mindspore import Tensor
from mindspore import numpy as np
from mindspore.ops import operations as P
import math
class PositionalEncoding(nn.Cell):

    def __init__(self,d_model, dropout=0.1, max_len=500) -> None:
        super().__init__()
        self.dropout = nn.Dropout(keep_prob=1-dropout)
        # https://www.mindspore.cn/docs/zh-CN/r2.0.0-alpha/note/api_mapping/pytorch_diff/mindspore.nn.Dropout.html?highlight=Dropout
        position = np.arange(max_len).reshape(max_len,1)
        div_term = ops.exp(np.arange(0, d_model, 2)*(-math.log(10000.0) / d_model))
        pe = np.zeros((max_len,d_model))
        pe[:, 0::2] = ops.sin(position * div_term)
        pe[:, 1::2] = ops.cos(position * div_term)
        self.pe=pe

    def construct(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, feature_num]
        """
        # pe with shape: 
        x = ops.add(x, self.pe[:x.shape[1]].unsqueeze(0))
        # 所有的操作都必须ops来搞
        return self.dropout(x)
    

max_len = 24 
d_model = 24

position = np.arange(max_len).reshape(max_len,1)
div_term = ops.exp(np.arange(0, d_model, 2)*(-math.log(10000.0) / d_model))
pe = np.zeros((max_len,d_model))
pe[:, 0::2] = ops.sin(position * div_term)
pe[:, 1::2] = ops.cos(position * div_term)
add_x = pe[:24].unsqueeze(0)
x = Tensor(np.ones((2,24,24)))
res_x = pe_test(x)

In [8]:

class Discriminator(nn.Cell): #D_y
    def __init__(self, auto_prefix=True, flags=None, in_features=24):
        super().__init__(auto_prefix, flags)
        self.in_features = in_features
        self.li = nn.SequentialCell(
            nn.Dense(in_features,512),
            # to config the detail
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dense(512,1),
            nn.Sigmoid()
        )
        self.alpha = 1 
    
    def construct(self, x):
        # 由于实现了自定义的算子，因此需要进行迁移
        # https://zhuanlan.zhihu.com/p/548702030 -> 直接在forward处进行修改
        # 前向传播的过程中使用了梯度翻转层的
        # 确定需求所在：https://www.mindspore.cn/tutorials/experts/zh-CN/r1.10/network/custom_cell_reverse.html
        # jump forward
        if x.shape[0] == 1:
            pad = np.zeros((1,self.in_features))
            # cuda?
            x = ops.concat((x,pad))
            # diff in concat https://www.mindspore.cn/docs/zh-CN/r1.9/note/api_mapping/pytorch_diff/Concat.html
            y = self.li(x)[0].unsqueeze(0)
            return y
        return self.li(x)

class backboneDiscriminator(nn.Cell): #D_f
    def __init__(self, seq_len, d=24):
        super().__init__()
        self.seq_len = seq_len 
        self.li1 = nn.Dense(seq_len, 512)
        self.li2 = nn.SequentialCell(
            nn.Dense(seq_len,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dense(512,1),
            nn.Sigmoid()
        )

    def construct(self,x):
        # jump forward
        out1 = self.li1(x).squeeze(2)
        if x.shape[0] ==1:
            pad = np.zeros((1,self.seq_len))
            out1 = ops.concat((out1,pad))
            out2 = self.li2(out1)[0].unsqueeze(0)
            return out2
        out2 = self.li2(out1)
        return out2
    
    





In [ ]:
max_len = 100
d_model = 20
pe = np.zeros((max_len,d_model))
position = np.arange(max_len).reshape(max_len,1)
div_term = ops.exp(np.arange(start=0, stop=d_model, step=2)*(-math.log(10000.0) / d_model))
pe[:, 0::2] = ops.sin(position * div_term)
pe[:, 1::2] = ops.cos(position * div_term)

In [ ]:
pe.unsqueeze(0)

Tensor(shape=[1, 100, 20], dtype=Float32, value=
[[[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00 ...  1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
  [ 8.41471016e-01,  5.40302277e-01,  3.87674242e-01 ...  9.99999821e-01,  2.51188554e-04,  9.99999940e-01],
  [ 9.09297466e-01, -4.16146815e-01,  7.14713454e-01 ...  9.99999225e-01,  5.02377108e-04,  9.99999881e-01],
  ...
  [ 3.79607737e-01, -9.25147533e-01,  7.93952167e-01 ...  9.98127699e-01,  2.43628789e-02,  9.99703169e-01],
  [-5.73381841e-01, -8.19288254e-01,  9.67561126e-01 ...  9.98088896e-01,  2.46139914e-02,  9.99697030e-01],
  [-9.99206841e-01,  3.98208797e-02,  9.89835501e-01 ...  9.98049736e-01,  2.48651039e-02,  9.99690831e-01]]])

In [ ]:
max_len = 500
position = torch.arange(max_len).unsqueeze(1)

In [ ]:
from mindspore import Tensor
import mindspore.numpy as np
position = np.arange(max_len).reshape(max_len,1)

In [ ]:
from mindspore import ops as ops
import math
div_term = ops.Exp(np.arange(0, d_model, 2)*(-math.log(10000.0) / d_model))


pe = np.zeros(max_len,d_model)

pe[:, 0::2] = ops.sin(position * div_term)
pe[:, 1::2] = ops.cos(position * div_term)



NameError: name 'np' is not defined

In [ ]:
import mindspore.numpy as np
print(np.zeros((2,2)))


a = np.zeros((2,2))

[[0. 0.]
 [0. 0.]]


In [ ]:
import mindspore.nn as nn
import mindspore.nn.transformer as trans
from mindspore.common import initializer
class mymodel(nn.Cell):

    def __init__(self, d_model = 24, dropout=0.1, nhead=8, nlayers=2, max_len=500):
        super().__init__()
        self.max_len = max_len
        self.pos_encoder = PositionalEncoding(d_model,nlayers)
        self.transformer_encoder = trans.TransformerEncoder(num_layers=nlayers,d_model=d_model,num_heads=nhead,ffn_hidden_size=512,dropout=0.1,post_layernorm_residual=True)
        self.d_model = d_model
        self.dropout = nn.Dropout(1-dropout)
        self.decoder = nn.Dense(d_model, 1 ,weight_init = initializer.Uniform(scale=0.1) )
        # bias default is zeros
        
    def construct(self, src, key_msk, attn_msk=None):
        src = self.pos_encoder(src)
        output1 = self.transformer_encoder(src, attn_msk, key_msk)
        output2 = self.decoder(output1)
        return output1, output2
    

In [2]:
from mydataset import TRANSFORMER_DATA_MINDS,TRANSFORMER_ALL_DATA_MINDS
from mymodel import mymodel, Discriminator, backboneDiscriminator
from tqdm import tqdm
import random
import numpy
import os 
import mindspore.dataset as ds
target= 'FD002'
source = 'FD003'
epoches = 240
os.chdir('/Domain-Adaptive-Remaining-Useful-Life-Prediction-with-Transformer/')
batch_size = 1000
seq_len = 70
source_list = numpy.loadtxt("save/"+source+"/train"+source+".txt", dtype=str).tolist()
target_list = numpy.loadtxt("save/"+target+"/train"+target+".txt", dtype=str).tolist()
valid_list = numpy.loadtxt("save/"+target+"/test"+target+".txt", dtype=str).tolist()
a_list = numpy.loadtxt("save/"+target+"/valid"+target+".txt", dtype=str).tolist()
target_test_names = valid_list + a_list
minl = min(len(source_list), len(target_list))
s_data = TRANSFORMER_ALL_DATA_MINDS(source_list, seq_len)
t_data = TRANSFORMER_ALL_DATA_MINDS(target_list, seq_len)
t_data_test = TRANSFORMER_ALL_DATA_MINDS(target_test_names, seq_len)


100%|██████████| 78/78 [00:12<00:00,  6.21it/s]


TypeError: For 'Dropout', the type of 'keep_prob' should be 'float', but got type 'int'.

In [3]:

# prepare model
net = mymodel(max_len=seq_len) 
D1 = Discriminator(seq_len)
D2 = backboneDiscriminator(seq_len)

TypeError: For 'Dropout', the type of 'keep_prob' should be 'float', but got type 'int'.